In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import gc
import os
from pathlib import Path
import shap
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import model_selection
import joblib
from multiprocessing import Pool
c_time = time.strftime("%Y%m%d_%H%M%S", time.localtime())
c_time_m = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
# 参数
# ======== System Setup ========
Version = 'V2.2sym'
EPOCH = 160
CORE_NUM = 32
# 确保EPOCH*REPEAT_ROUND是CORE_NUM的整数倍
TRAIN_TEST_SPLIT = 0.85
# ======== Fit Data Input ========
S_N = 43
F_N = 62
INPUT_X = 'Features_'+str(S_N)+'_'+str(F_N)+'.csv'
INPUT_Y = 'Values_True_'+str(S_N)+'.csv'
INPUT_TITLE = 'Title_'+str(F_N)+'.csv'
INPUT_SMILES = 'Smiles_'+str(S_N)+'.csv'
RECORD_NAME = 'Record_Stepreg_'+Version+'_'+c_time+'.txt'

X = np.loadtxt(INPUT_X, delimiter=',')
y = np.loadtxt(INPUT_Y)
title = np.loadtxt(INPUT_TITLE, dtype=str, delimiter=',', comments='!')
#是否设定第一特征

# List of test set indices provided by the user
test_indices = [5,9,23,35]  # 对应是Full的matrix是刚刚好（有title的特征和标签值的Full）
test_indices = [i - 2 for i in test_indices]
test_mask = np.zeros(len(y), dtype=bool)
test_mask[test_indices] = True
X_train = X[~test_mask]
y_train = y[~test_mask]
Xrealtest = X[test_mask]
yrealtest = y[test_mask]
X=X_train
y=y_train
import os
from pathlib import Path
DIR = 'QuickRegression_'+Version+'_FindSplit_'+str(X.shape[1])+'_Fs_'+c_time
print(DIR)
os.mkdir(DIR)
point = round(X.shape[0]*TRAIN_TEST_SPLIT)


QuickRegression_V2.2sym_FindSplit_62_Fs_20250309_164848


In [2]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from multiprocessing import Pool
from pathlib import Path
import matplotlib.pyplot as plt
import time
import joblib
import os

# 定义模型名称和对应的类
MODELS = {
    "Lasso": Lasso,
    "GBRT": GradientBoostingRegressor,
    "XGBoost": XGBRegressor,
    "Elastic": ElasticNet,
    "DecisionTree": DecisionTreeRegressor,
    "RandomForest": RandomForestRegressor,
    "Adaboost": AdaBoostRegressor,
    "Catboost": CatBoostRegressor
}


# 定义目标函数（超参数优化）
def objective(trial, model_name, X, y):
    if model_name == "Lasso":
        params = {
            'alpha': trial.suggest_float('alpha', 1e-5, 1.0, log=True),
            'max_iter': trial.suggest_int('max_iter', 100, 1000)
        }
        model = MODELS[model_name](**params)
    elif model_name == "GBRT":
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0)
        }
        model = MODELS[model_name](**params)
    elif model_name == "XGBoost":
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1.0, log=True),'n_jobs': 1,
        }
        model = MODELS[model_name](**params)
    elif model_name == "Elastic":
        params = {
            'alpha': trial.suggest_float('alpha', 1e-5, 1.0, log=True),
            'l1_ratio': trial.suggest_float('l1_ratio', 0.1, 1.0)
        }
        model = MODELS[model_name](**params)
    elif model_name == "DecisionTree":
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
        }
        model = MODELS[model_name](**params)
    elif model_name == "RandomForest":
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
        }
        model = MODELS[model_name](**params)
    elif model_name == "Adaboost":
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1)
        }
        model = MODELS[model_name](**params)
    elif model_name == "Catboost":
        params = {
            'iterations': trial.suggest_int('iterations', 100, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'depth': trial.suggest_int('depth', 3, 10)
        }
        model = MODELS[model_name](**params, verbose=0)
    
    # 划分训练集和验证集
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 预测验证集
    y_pred = model.predict(X_val)
    
    # 计算损失
    mse = mean_squared_error(y_val, y_pred)
    return mse

# 通用模型训练函数
def Model_Fit(model_name, X, y, X_train, y_train, X_test, y_test, paras):
    model = MODELS[model_name]()
    for k, v in paras.items():
        if hasattr(model, k):  # 确保参数有效
            model.set_params(**{k: v})
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 预测
    y_pred = model.predict(X_test)
    
    # 计算指标
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    temp = [mse, mae, r2]
    print(f'   {model_name} MSE: {mse:.5f}  MAE: {mae:.5f}  R^2: {r2:.5f}')
    
    return (temp, model)

# 多进程交叉验证
def poolfit(model_name, TRAIN_TEST_SPLIT, EPOCH, CORE_NUM, X, y, paras, Xrealtest):
    r_l = []
    split_l = []
    test_idx_m = []
    point = round(X.shape[0] * TRAIN_TEST_SPLIT)
    
    for _ in range(int(EPOCH / CORE_NUM)):
        print(f'Round {CORE_NUM*(_)+1} Begin:')
        pool = Pool(CORE_NUM)
        for __ in range(CORE_NUM):
            permutation = np.random.permutation(y.shape[0])
            train_idx = permutation[:point]
            test_idx = permutation[point:]
            X_train = X[train_idx, :]
            y_train = y[train_idx]
            X_test = X[test_idx, :]
            y_test = y[test_idx]
            split_l.append(train_idx)
            test_idx_m.append(test_idx)
            r = pool.apply_async(Model_Fit, args=(model_name, X, y, X_train, y_train, X_test, y_test, paras))
            r_l.append(r)
        pool.close()
        pool.join()

    # 收集结果
    full_m = np.zeros((len(r_l), X.shape[0]))
    y_realtest_pred_list = []
    mse_list, mae_list, r2_list = [], [], []
    
    for i in range(len(r_l)):
        results = r_l[i].get()
        temp = results[0]
        clf_new = results[1]
        
        mse_list.append(temp[0])
        mae_list.append(temp[1])
        r2_list.append(temp[2])
        
        y_full_pred = clf_new.predict(X)
        y_realtest_pred = clf_new.predict(Xrealtest)
        y_realtest_pred_list.append(y_realtest_pred)
        full_m[i] = y_full_pred

    # 统计指标
    mse_mean = np.mean(mse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    mse_std = np.std(mse_list)
    y_realtest_pred_std = np.std(y_realtest_pred_list, axis=0)
    
    # 保存预测结果
    y_realtest_pred = np.mean(y_realtest_pred_list, axis=0)
    np.savetxt(Path(DIR, f'{model_name}_All_Realtestpred_{c_time}.txt'), y_realtest_pred, fmt='%.5f')
    
    # 生成可视化图表
    generate_plots(model_name, X, y, full_m, test_idx_m, 
                   (mse_mean, mae_mean, r2_mean, mse_std, y_realtest_pred, y_realtest_pred_std))
    
    return (mse_mean, mae_mean, r2_mean, full_m, test_idx_m, mse_std, mse_std, y_realtest_pred, y_realtest_pred_std)

# 生成可视化图表
def generate_plots(model_name, X, y, full_m, test_idx_m, metrics):
    test_data_m = [[] for _ in range(X.shape[0])]
    for i in range(full_m.shape[0]):
        for j in test_idx_m[i]:
            test_data_m[j].append(full_m[i, j])
    
    test_mean_l = [np.mean(x) for x in test_data_m]
    test_std_l = [np.std(x) for x in test_data_m]
    true_y = y.flatten()
    
    # 绘制散点图
    fig = plt.figure(figsize=(10, 8), dpi=300)
    ax = fig.add_axes([0.11, 0.08, 0.88, 0.815])
    sc = ax.scatter(true_y, test_mean_l, alpha=0.55, c=test_std_l, cmap='viridis', marker='o')
    
    # 绘制参考线
    left_limit = min(true_y.min()-1, np.array(test_mean_l).min()-1)
    right_limit = max(true_y.max()+1, np.array(test_mean_l).max()+1)
    ax.plot([left_limit, right_limit], [left_limit, right_limit], color='#B22222', linestyle=':', linewidth=2)
    ax.plot([left_limit, right_limit], [left_limit+1, right_limit+1], color='#FFA500', linestyle=':', linewidth=2)
    ax.plot([left_limit, right_limit], [left_limit-1, right_limit-1], color='#FFA500', linestyle=':', linewidth=2)
    
    # 添加标签和图例
    ax.legend(['Correct', 'Correct+1', 'Correct-1', 'Mean Prediction'], loc='upper left', fontsize=14)
    ax.set_xlabel('True Values', fontsize=14)
    ax.set_ylabel('Predicted Values', fontsize=14)
    plt.title(f'{model_name} Performance: MSE={metrics[0]:.4f}, MAE={metrics[1]:.4f}, R²={metrics[2]:.4f}', fontsize=16)
    
    # 保存图表
    plt.savefig(Path(DIR, f'{model_name}_Prediction_Distribution_{c_time}.png'))
    plt.close()
    xgbscatterdata= np.column_stack((true_y, test_mean_l, test_std_l))
    save_name1 =f'{model_name}_Prediction_Distribution_{c_time}.txt'
    save_name2 = Path('.', DIR, save_name1)
    with open(save_name2,"w") as f:
        np.savetxt(save_name2,xgbscatterdata, fmt='%s', delimiter=',', comments='!')

# 使用示例
if __name__ == "__main__":
    # 假设有以下模拟数据
    X = X
    y = y
    Xrealtest = Xrealtest
    n_trials = 200
       # 创建输出文件路径
    output_file = Path(DIR, f'training_log_{c_time}.txt')
    
    # 打开文件并写入参数设置和训练信息
    with open(output_file, 'w') as f:
        # 写入参数设置
        f.write("# 参数设置\n")
        f.write(f"TRAIN_TEST_SPLIT = {TRAIN_TEST_SPLIT}\n")
        f.write(f"EPOCH = {EPOCH}\n")
        f.write(f"CORE_NUM = {CORE_NUM}\n\n")
        f.write(f"n_trials  = {n_trials }\n\n")
        f.write(f"INPUT_X = 'Features_{S_N}_{F_N}.csv'\n")
        f.write(f"INPUT_Y = 'Values_True_{S_N}.csv'\n")
        f.write(f"INPUT_TITLE = 'Title_{F_N}.csv'\n")
        f.write(f"INPUT_SMILES = 'Smiles_{S_N}.csv'\n")
        f.write(f"test_indices = {test_indices}\n\n")
    # 遍历所有模型
        for model_name in MODELS.keys():
            print(f"\n{'='*30} Training {model_name} {'='*30}")
            # 超参数优化
            study = optuna.create_study(direction='minimize')
            study.optimize(lambda trial: objective(trial, model_name, X, y), n_trials=n_trials)
            best_params = study.best_params
            print(f"Best Params for {model_name}: {best_params}")
            f.write(f"Best Params for {model_name}: {best_params}\n")
            # 创建最终模型
            final_model = MODELS[model_name](**best_params)
            # 交叉验证和结果保存
            paras = final_model.get_params()
            perf = poolfit(model_name, TRAIN_TEST_SPLIT, EPOCH, CORE_NUM, X, y, paras, Xrealtest)    


[I 2025-03-09 16:48:48,798] A new study created in memory with name: no-name-b87e9a06-7ac3-4af1-8351-e78efe26c7fe
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.310e-02, tolerance: 3.847e-04
[I 2025-03-09 16:48:48,800] Trial 0 finished with value: 1.5458238415563097 and parameters: {'alpha': 1.6464352365208447e-05, 'max_iter': 313}. Best is trial 0 with value: 1.5458238415563097.
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.584e-02, tolerance: 3.847e-04
[I 2025-03-09 16:48:48,801] Trial 1 finished with value: 3.0400367274036277 and parameters: {'alpha': 1.3472023265494353e-05, 'max_iter': 129}. Best is trial 0 with value: 1.5458238415563097.
Objective did not converge. You might want to increase the number of iterations, check the scale of the feat

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.963e-01, tolerance: 3.847e-04
[I 2025-03-09 16:48:48,906] Trial 27 finished with value: 0.05985803504889536 and parameters: {'alpha': 0.020345137785328118, 'max_iter': 733}. Best is trial 18 with value: 0.027391526286255168.
[I 2025-03-09 16:48:48,912] Trial 28 finished with value: 0.0353542223923722 and parameters: {'alpha': 0.32464806936757157, 'max_iter': 614}. Best is trial 18 with value: 0.027391526286255168.
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.876e-03, tolerance: 3.847e-04
[I 2025-03-09 16:48:48,917] Trial 29 finished with value: 0.03511213712735589 and parameters: {'alpha': 0.11273541994231323, 'max_iter': 481}. Best is trial 18 with value: 0.027391526286255168.
Objective did not conver


============================== Training Lasso ==============================


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.914e-02, tolerance: 3.847e-04
[I 2025-03-09 16:48:48,966] Trial 38 finished with value: 0.04259430200138003 and parameters: {'alpha': 0.07186051844486091, 'max_iter': 578}. Best is trial 18 with value: 0.027391526286255168.
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.512e-01, tolerance: 3.847e-04
[I 2025-03-09 16:48:48,971] Trial 39 finished with value: 0.11858475522841057 and parameters: {'alpha': 0.0021612639526422126, 'max_iter': 149}. Best is trial 18 with value: 0.027391526286255168.
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.873e-02, tolerance: 3.847e-04
[I 2025-03-09 16:48

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.562e-03, tolerance: 3.847e-04
[I 2025-03-09 16:48:49,138] Trial 67 finished with value: 0.0348975590431689 and parameters: {'alpha': 0.6028197931636111, 'max_iter': 536}. Best is trial 66 with value: 0.026726010712301115.
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.954e-02, tolerance: 3.847e-04
[I 2025-03-09 16:48:49,145] Trial 68 finished with value: 0.030931736270462775 and parameters: {'alpha': 0.19179507642397112, 'max_iter': 346}. Best is trial 66 with value: 0.026726010712301115.
[I 2025-03-09 16:48:49,151] Trial 69 finished with value: 0.03094197574922533 and parameters: {'alpha': 0.2741753516205867, 'max_iter': 477}. Best is trial 66 with value: 0.026726010712301115.
[I 2025-03-09 16:48:49,157

...

856:	learn: 0.0013202	total: 1.73s	remaining: 289ms
867:	learn: 0.0011976	total: 1.74s	remaining: 265ms
860:	learn: 0.0013551	total: 1.74s	remaining: 281ms
854:	learn: 0.0013182	total: 1.72s	remaining: 291ms
857:	learn: 0.0014720	total: 1.72s	remaining: 285ms
862:	learn: 0.0013841	total: 1.73s	remaining: 275ms
866:	learn: 0.0012921	total: 1.73s	remaining: 265ms
862:	learn: 0.0014471	total: 1.72s	remaining: 274ms
855:	learn: 0.0014235	total: 1.73s	remaining: 291ms
866:	learn: 0.0013850	total: 1.73s	remaining: 266ms
863:	learn: 0.0014221	total: 1.73s	remaining: 272ms
867:	learn: 0.0014998	total: 1.75s	remaining: 266ms
869:	learn: 0.0014199	total: 1.74s	remaining: 261ms
856:	learn: 0.0014990	total: 1.73s	remaining: 289ms
867:	learn: 0.0013516	total: 1.74s	remaining: 265ms
851:	learn: 0.0016577	total: 1.72s	remaining: 299ms
862:	learn: 0.0014297	total: 1.72s	remaining: 273ms
862:	learn: 0.0012930	total: 1.73s	remaining: 275ms
857:	learn: 0.0015587	total: 1.75s	remaining: 289ms
866:	learn: 

867:	learn: 0.0012514	total: 1.74s	remaining: 265ms
865:	learn: 0.0013299	total: 1.74s	remaining: 269ms
871:	learn: 0.0014610	total: 1.75s	remaining: 258ms
861:	learn: 0.0015268	total: 1.76s	remaining: 281ms
867:	learn: 0.0013467	total: 1.74s	remaining: 265ms
874:	learn: 0.0013740	total: 1.75s	remaining: 250ms
855:	learn: 0.0013528	total: 1.74s	remaining: 293ms
876:	learn: 0.0013366	total: 1.75s	remaining: 246ms
876:	learn: 0.0013789	total: 1.75s	remaining: 246ms
870:	learn: 0.0013743	total: 1.74s	remaining: 258ms
865:	learn: 0.0013139	total: 1.75s	remaining: 271ms
860:	learn: 0.0013867	total: 1.74s	remaining: 281ms
869:	learn: 0.0013891	total: 1.75s	remaining: 262ms
872:	learn: 0.0011592	total: 1.75s	remaining: 255ms
871:	learn: 0.0012534	total: 1.74s	remaining: 255ms
867:	learn: 0.0014053	total: 1.73s	remaining: 264ms
872:	learn: 0.0009596	total: 1.76s	remaining: 256ms
861:	learn: 0.0012797	total: 1.75s	remaining: 279ms
867:	learn: 0.0013743	total: 1.73s	remaining: 263ms
857:	learn: 

972:	learn: 0.0006988	total: 1.95s	remaining: 54.2ms
983:	learn: 0.0007488	total: 1.96s	remaining: 31.9ms
980:	learn: 0.0007200	total: 1.97s	remaining: 38.1ms
969:	learn: 0.0008032	total: 1.94s	remaining: 60.1ms
964:	learn: 0.0007043	total: 1.96s	remaining: 71.1ms
976:	learn: 0.0004892	total: 1.98s	remaining: 46.5ms
980:	learn: 0.0006806	total: 1.97s	remaining: 38.1ms
987:	learn: 0.0007586	total: 1.98s	remaining: 24.1ms
975:	learn: 0.0007002	total: 1.98s	remaining: 48.6ms
989:	learn: 0.0006827	total: 1.98s	remaining: 20ms
978:	learn: 0.0007305	total: 1.96s	remaining: 42ms
974:	learn: 0.0007448	total: 1.95s	remaining: 49.9ms
983:	learn: 0.0006574	total: 1.98s	remaining: 32.2ms
977:	learn: 0.0007383	total: 1.95s	remaining: 44ms
967:	learn: 0.0008113	total: 1.95s	remaining: 64.6ms
987:	learn: 0.0004672	total: 1.99s	remaining: 24.1ms
981:	learn: 0.0007263	total: 1.97s	remaining: 36.2ms
981:	learn: 0.0006319	total: 1.96s	remaining: 36ms
971:	learn: 0.0006224	total: 1.96s	remaining: 56.3ms
9

986:	learn: 0.0006082	total: 1.98s	remaining: 26.1ms
985:	learn: 0.0006169	total: 1.97s	remaining: 28ms
983:	learn: 0.0006492	total: 1.97s	remaining: 32ms
982:	learn: 0.0007153	total: 1.96s	remaining: 34ms
985:	learn: 0.0006987	total: 1.98s	remaining: 28.1ms
980:	learn: 0.0006793	total: 1.99s	remaining: 38.5ms
983:	learn: 0.0007088	total: 1.97s	remaining: 32ms
988:	learn: 0.0007255	total: 1.97s	remaining: 22ms
980:	learn: 0.0007195	total: 1.96s	remaining: 37.9ms
992:	learn: 0.0004532	total: 2s	remaining: 14.1ms
979:	learn: 0.0006995	total: 1.97s	remaining: 40.3ms
990:	learn: 0.0005868	total: 1.98s	remaining: 18ms
985:	learn: 0.0006584	total: 1.97s	remaining: 28ms
994:	learn: 0.0006627	total: 1.99s	remaining: 9.98ms
977:	learn: 0.0007018	total: 1.97s	remaining: 44.4ms
977:	learn: 0.0006787	total: 1.96s	remaining: 44.2ms
976:	learn: 0.0006075	total: 1.97s	remaining: 46.3ms
969:	learn: 0.0006828	total: 1.97s	remaining: 60.9ms
992:	learn: 0.0007371	total: 1.99s	remaining: 14.1ms
994:	learn

985:	learn: 0.0007006	total: 1.97s	remaining: 27.9ms
977:	learn: 0.0006453	total: 1.98s	remaining: 44.7ms
987:	learn: 0.0006930	total: 1.97s	remaining: 24ms
984:	learn: 0.0006796	total: 1.98s	remaining: 30.2ms
974:	learn: 0.0006604	total: 1.98s	remaining: 50.8ms
976:	learn: 0.0007680	total: 1.97s	remaining: 46.4ms
983:	learn: 0.0006890	total: 1.98s	remaining: 32.3ms
997:	learn: 0.0004392	total: 2.01s	remaining: 4.02ms
990:	learn: 0.0006366	total: 1.98s	remaining: 18ms
992:	learn: 0.0006728	total: 1.99s	remaining: 14ms
992:	learn: 0.0005856	total: 1.99s	remaining: 14ms
996:	learn: 0.0007211	total: 2s	remaining: 6.03ms
988:	learn: 0.0006265	total: 1.98s	remaining: 22ms
981:	learn: 0.0005871	total: 1.97s	remaining: 36.2ms
990:	learn: 0.0005987	total: 1.98s	remaining: 18ms
988:	learn: 0.0006499	total: 1.98s	remaining: 22.1ms
999:	learn: 0.0006435	total: 2s	remaining: 0us
986:	learn: 0.0004618	total: 2s	remaining: 26.3ms
980:	learn: 0.0007513	total: 1.96s	remaining: 38.1ms
988:	learn: 0.000